# Kaggle League of Legends competition - Sample Submission

## Team: Elden Ring

<img src="https://eldenring.wiki.fextralife.com/file/Elden-Ring/mirel_pastor_of_vow.jpg" alt="PRAISE DOG" style="width:806px;height:600px;"/>

#### PRAISE THE DOG!

In [27]:
import pandas as pd

In [28]:
X_train = pd.read_csv('../data/participants_train.csv')
X_test = pd.read_csv('../data/participants_test.csv')
y_train = pd.read_csv('../data/train_winners.csv')
submission = pd.read_csv('../data/sample_submission.csv')

In [29]:
X_train

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId
0,0,100,1,0,303,Mordekaiser,82
1,0,100,2,1,616,Sylas,517
2,0,100,3,2,667,Lissandra,127
3,0,100,4,3,860,Caitlyn,51
4,0,100,5,4,325,Morgana,25
...,...,...,...,...,...,...,...
79995,7999,200,6,13979,595,Yorick,83
79996,7999,200,7,39643,38,Volibear,106
79997,7999,200,8,5570,498,Anivia,34
79998,7999,200,9,10228,733,Twitch,29


I'll make a really simple model - I'll simply predict the winner based on which team has the highest level summoner.

In [30]:
train_predictions = ( 
    X_train
    .sort_values(['matchId', 'summonerLevel'], ascending = [True, False])
    .drop_duplicates('matchId')
    [['matchId', 'teamId']]
    .reset_index(drop = True)
)

train_predictions

,matchId,teamId
0,0,100
1,1,100
2,2,200
3,3,200
4,4,100
...,...,...
7995,7995,200
7996,7996,200
7997,7997,100
7998,7998,200


A quick idea that comes to mind is to instead of taking just a single summoner level, take all 5 in the team and sum them.

In [57]:
train_predictions2 =(
    X_train
    .groupby(['matchId', 'teamId'])[['summonerLevel']].sum()
    .reset_index()
    .sort_values(['matchId', 'summonerLevel'], ascending=[True, False])
    .drop_duplicates('matchId')
    [['matchId', 'teamId']]
    .reset_index(drop = True)
)

train_predictions2

,matchId,teamId
0,0,100
1,1,100
2,2,100
3,3,200
4,4,100
...,...,...
7995,7995,100
7996,7996,200
7997,7997,100
7998,7998,200


In [31]:
from sklearn.metrics import accuracy_score

In [32]:
accuracy_score(
    y_true = y_train['winner'],
    y_pred = train_predictions['teamId']
)

0.504

In [58]:
accuracy_score(
    y_true = y_train['winner'],
    y_pred = train_predictions2['teamId']
)

0.497125

Looks like I do a pretty terrible job with this method. Nevertheless, I'll implement this on the test data.

> note: it appears that summing the summoner lvl yields even worse results!

In [33]:
y_pred = ( 
    X_test
    .sort_values(['matchId', 'summonerLevel'], ascending = [True, False])
    .drop_duplicates('matchId')
    ['teamId']
    .reset_index(drop = True)
)

y_pred

0       100
1       100
2       200
3       200
4       200
       ... 
1995    100
1996    200
1997    200
1998    200
1999    100
Name: teamId, Length: 2000, dtype: int64

I'll add these to the sample submission.

In [34]:
submission['winner'] = y_pred
submission.head()

,matchId,winner
0,8000,100
1,8001,100
2,8002,200
3,8003,200
4,8004,200


Now, I'll export the result.

In [11]:
submission.to_csv('../data/first_submission.csv', index = False)